<a href="https://colab.research.google.com/github/kurota0612/test/blob/main/GitTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 引き当てたGPUの情報を確認する
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



##各種設定

In [1]:
import os

# Google Cloud SDKの権限許可
try:
    from google.colab import auth
    auth.authenticate_user()
except ImportError:
    pass

# GitHubのリポジトリ名
REPOSITORY_NAME = '20th_Beginner'
# GCEのプロジェクト名
PROJECT_ID = 'kaggleops-project'
# GCSのバケット名
BUCKET_NAME = 'kaggleops-bucket'

# 環境変数を設定してる
# 左辺は辞書。"GCLOUD_PROJECT"はkey。そのvalueとして「PROJECT_ID」
os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [2]:
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [30]:
# Colabのローカルにリポジトリをclone
!git clone https://github.com/kurota0612/{REPOSITORY_NAME}.git

Cloning into '20th_Beginner'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 119 (delta 38), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (119/119), 41.67 KiB | 8.33 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [4]:
# パッケージの一括インストール
!pip install -qr ./{REPOSITORY_NAME}/requirements.txt

     |████████████████████████████████| 11.9 MB 33.3 MB/s 
     |████████████████████████████████| 563 kB 50.2 MB/s 
     |████████████████████████████████| 6.0 MB 22.9 MB/s 
     |████████████████████████████████| 146 kB 51.3 MB/s 
     |████████████████████████████████| 181 kB 32.7 MB/s 
     |████████████████████████████████| 346 kB 40.6 MB/s 
     |████████████████████████████████| 79 kB 5.0 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
     |████████████████████████████████| 210 kB 37.8 MB/s 
     |████████████████████████████████| 1.2 MB 40.0 MB/s 
     |████████████████████████████████| 132 kB 54.7 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
     |████████████████████████████████| 5.6 MB 47.0 MB/s 
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 3.6 MB 44.2 MB/s 
     |████████████████████████████████| 179 kB 29.1 MB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |███████████████

In [31]:
# 現在のディレクトリを変更
# https://qiita.com/bear_montblanc/items/64d7efd9e4ea7caa98f9
%cd ./{REPOSITORY_NAME}/mlflow

/content/20th_Beginner/20th_Beginner/mlflow


In [ ]:
# Google Cloundの設定
!gcloud config set project {PROJECT_ID}

In [ ]:
# !gsutil -mq cp -rn gs://{BUCKET_NAME}/mlruns /content/{BUCKET_NAME}/src

# 
blobs = storage_client.list_blobs(BUCKET_NAME)
for blob in blobs:
    filepath = blob.name
    if filepath[:7]=="mlruns/" and len(filepath)!=7:
        os.makedirs("/".join(filepath.split("/")[:-1]), exist_ok=True)
        blob.download_to_filename(filepath)

##プログラム実行

mlproject.pyの中でMLprojectファイル（YAML？）を実行してる

⇨MLprojectファイルの中でtrain.pyを実行してる

⇨train.pyが実際の学習コード

In [35]:
!python mlproject.py

Traceback (most recent call last):
  File "mlproject.py", line 8, in <module>
    with open(os.path.join(FILE_DIR, "./config/config.yml")) as file:
FileNotFoundError: [Errno 2] No such file or directory: '/content/20th_Beginner/20th_Beginner/mlflow/./config/config.yml'


##ローカルに保存されたmlrunsをGCSに転送

ローカル（Colab？）に保存されたmlflowの結果ファイルをGCSに転送する

⇨後でGCSからGCEに結果ファイルをコピーしてブラウザから結果を確認する

In [ ]:
# !gsutil -mq cp -rn /content/{REPOSITORY_NAME}/src/mlruns gs://{BUCKET_NAME}

import glob

files = [f for f in glob.glob("mlruns/**", recursive=True) if f[:7]=="mlruns/" and len(f)!=7]
files_in_bucket = [f.name for f in storage_client.list_blobs(BUCKET_NAME)]
bucket = storage_client.get_bucket(BUCKET_NAME)

for f in files:
    try:
        if f not in files_in_bucket:
            blob = bucket.blob(f)
            blob.upload_from_filename(f)
    except:
        pass